# Step Options

## Step options

**Step options** are specified after step name that assists the specification of workflows. SoS provides the following options

### Option `skip`

Option `skip` takes two formats, the first format has no value

```
[10: skip]
```

and is equivalent to

```
[10: skip=True]
```

The whole step will be skipped as if it is not defined at all in the script. This option provides a quick method to disable a step.

The second format takes a value, which is usually an expression that will be evaluated when the step is executed. For example, step 10 is by default executed by default

In [5]:
parameter: qc = True
[10 (quality check): skip=not qc]
print("${step_name} is executed")

default_10 is executed


but will not be executed if the workflow is executed with option `--no-qc` (for `qc=False`)

In [7]:
%run --no-qc
parameter: qc = True
[10 (quality check): skip=not qc]
print("${step_name} is executed")

### Option `sigil`

Option `sigil` accepts a string for an alternative sigil, or `None` to disable string interpolation in the step. The sigil must be two strings separated by a space, such as `%( )`, `< >`, and `#{ }`. Sigils with equal left and right symbol such as `# #` can be used although they do not support features such as nested interpolation.

### Option `shared`

SoS executes each step in a separate process and by default does not return any result to the master SoS process. Option `shared` is used to share variables between steps. This option accepts:

* A string (variable name), or
* A map between variable names and expressions (strings) that will be evaluated upon the completion of the step.
* A sequence of strings (variables) or maps.

After the completion of a step, local variables defined in the `shared` option will be sent to the global environment so that later steps can use the updated value. For example,

In [ ]:
parameter: trim = False
parameter: input_files = list

[10: shared={'input_files':'output'}, skip=trim is False]
input:  input_files, group_by=1
output: '${_input[0]}.trimmed'

sh:
    trim ${input} > ${output}

[20]
input: input_files
# do something

In this example, `trim` is an option to trim input files defined in another parameter `input_files`. If `trim` is false, step 10 is ignored and step 20 handles `input_files`. If `trim` is True, the files are trimmed and produces `${_input[0]}.trimmed`. Variable `input_files` are sent back to the global environment so that step 20 would be handling the trimmed version of input files.

A map syntax is recommended to share `output` of one step with others, because the variable assignment will be evaluated only after the step is complete:

In [ ]:
[1: shared = {'test_output': 'output'}]
...
[2]
input: test_output

The map syntax is evaluated as expressions; therefore it is possible to finer control what specific output, or variations of output, to share with others. For example:

In [ ]:
[1: shared={'test_output_1':'output[0]', 'test_output_2': 'output[1]'}]
...
[2]
input: test_output_1

to shared the first file in `output` (filename `output[0]`) instead of the entire output file list.

NOTE: Because global variables are readonly, they cannot be changed by individual steps through the `shared` option. Shared variable therefore has to be either new variables or parameters.

The `shared` option also provides a `sos_variable` target. Please refer to section [auxiliary step](Auxiliary_steps.html) for details.

### Option `provides`

This option lists files or targets a step generates so that it can be called if the target is required but does not exist. Please see section [auxiliary step](Auxiliary_Steps.html) for details. A step can be treated as both a forward step and an auxiliary step despite of the differences on how they are invoked.